# $\Phi$-SO demo : Encoding and decoding expressions


## Quick Start

### Library of choosable tokens

### Encoding mathematical expressions

### Decoding mathematical expressions

## For loop : Insert your ML model here 👇

## Features

### Priors

### Evaluating Expressions

### Optimizing free constants

### Expressions are auto-differentiable

### Sympy export

### Tree structure

### Physical units

### Class Expression : Exploiting multiple datasets